<a href="https://colab.research.google.com/github/shelleyg-bit/canada-land-cover-classifier/blob/main/gee_download_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies 

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install geopandas
!pip install geojson
!pip install geemap
!pip install geotable

In [18]:
%cd /content/drive/MyDrive/FarmHand

/content/drive/MyDrive/FarmHand


In [ ]:
!git status

On branch data_collection
Your branch is up to date with 'origin/data_collection'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   data_prep/gee_download_images.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [19]:
import ee
ee.Authenticate() # give this notebook access to your google ee account
ee.Initialize() # this talks to ee backend to find out about all features of ee

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nKGGdOOYE45f91eQii-zLCtljyIbY_ODys0BSY-mF-c&tc=rvKAkci6-7ImTlRMW1pf9QzeSUX3lnmWqBK8JdlTwXw&cc=6NaqMrZ1nDcem9OSym8VDnLJNaF-1VmdUklRGCJ6chg

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgx3NMaAfRGpxZgg3luYLDs0yHFVSPe55yEb9qxNc6UTc59BrNPEyk

Successfully saved authorization token.


In [20]:
import sys
sys.path.append('/content/drive/MyDrive/FarmHand/data_prep')

In [21]:
path1 ='/content/drive/MyDrive/FarmHand/data/All MB kml files/original_kmz_files/'
path2 ='/content/drive/MyDrive/farmhand/Data/farms_sheet.xlsx'
sheet ='Farm data (for farms created in'
path3 ='/content/drive/MyDrive/farmhand/Data/Mint farm data (2022 season)/Farmer data of mint. (1).xlsx'

In [22]:
from Kmz_n_excel_comb_2shp import comb_kmz_n_excel_2shp

df = comb_kmz_n_excel_2shp(path1, path2, sheet, path3)

shape file stored at the following location /content/drive/MyDrive/farmhand/Data/Mint farm data (2022 season) 


In [23]:
df.head()

,Name,geometry,class
0,K Shankar guru,"POLYGON Z ((76.73941 15.24347 0.00000, 76.7394...",Chilli-Non Farmhand
1,siddappa,"POLYGON Z ((76.71973 15.26482 0.00000, 76.7195...",Chilli-Non Farmhand
2,chandru nayaka,"POLYGON Z ((76.96205 15.12738 0.00000, 76.9616...",Chilli-Non Farmhand
3,Dodda Basava G,"POLYGON Z ((76.72902 15.24342 0.00000, 76.7290...",Chilli-Non Farmhand
4,udaya m,"POLYGON Z ((76.96406 15.12368 0.00000, 76.9636...",Chilli-Non Farmhand


In [33]:
from shapely.geometry import Polygon

# Remove the z value from each point in polygon
for i, boundary in enumerate(df['geometry'].values):
  coords = list(boundary.exterior.coords)
  df['geometry'][i] = Polygon([xyz[0:2] for xyz in coords])

  

In [34]:
df.head()

,Name,geometry,class
0,K Shankar guru,"POLYGON ((76.73941 15.24347, 76.73942 15.24347...",Chilli-Non Farmhand
1,siddappa,"POLYGON ((76.71973 15.26482, 76.71954 15.26474...",Chilli-Non Farmhand
2,chandru nayaka,"POLYGON ((76.96205 15.12738, 76.96169 15.12735...",Chilli-Non Farmhand
3,Dodda Basava G,"POLYGON ((76.72902 15.24342, 76.72901 15.24344...",Chilli-Non Farmhand
4,udaya m,"POLYGON ((76.96406 15.12368, 76.96365 15.12385...",Chilli-Non Farmhand


In [30]:
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [31]:
import geemap
import geopandas as gpd

import numpy as np

import matplotlib.pyplot as plt


## Create an EE Geometry Object for Area of Interest 



In [32]:
#Carefully choosen post analyzing
bbox1 =[76.65, 15.2,76.8, 15.37 ] #chilli farms aoi_1
bbox2 =[76.9, 15.05, 77.05, 15.15] #chilli farms aoi_2
bbox3 =[79.1996545, 28.4018788, 89.6, 28.75 ] #mint farms aoi_3

In [205]:
from shapely.geometry import box, Polygon, MultiPolygon

chilli_bbox1 = Polygon(box(*bbox1))
chilli_bbox2 = Polygon(box(*bbox2))
mint_bbox = Polygon(box(*bbox3))
aoi_gdf = gpd.GeoDataFrame({'geometry': [chilli_bbox1, chilli_bbox2, mint_bbox]}, crs=df.crs)
aoi = geemap.geopandas_to_ee(aoi_gdf)

In [204]:

aoi = geemap.geopandas_to_ee(aoi_gdf)
type(aoi.first())
aoi.first().getInfo()


{'geometry': {'coordinates': [[[76.65, 15.2],
    [76.8, 15.2],
    [76.8, 15.37],
    [76.65, 15.37],
    [76.65, 15.2]]],
  'type': 'Polygon'},
 'id': '0',
 'properties': {},
 'type': 'Feature'}

In [195]:
aoi.get(0).getInfo()
aoi_geom = ee.Geometry(aoi.get(0).getInfo()['geometry'])

In [196]:
aoi_geom.getInfo()

{'coordinates': [[[76.65, 15.2],
   [76.8, 15.2],
   [76.8, 15.37],
   [76.65, 15.37],
   [76.65, 15.2]]],
 'type': 'Polygon'}


## Get Image Collection from Sentinel

- For each harvest cycle(Sept-Feb) from 2019-2022, filtering out only harvest cycle months and discarding non-usable months (May-June)
- Collecting all the remaining raw bands on sentinel as @Mani Babu suggested. (right now I just collected 10m bands (RGB+NearIR) & NDVI)
- Combining All the bands for each harvest cycle in a single Tiff file.
- Monthly mean of every band
- A single tif will contain MonthName_Band_No as one band , so total number of bands will be  in one tiff will be Number of Months* Number of bands 
- Resulting in One GeoTiff file per harvest cycle
- Adding Mint farms to the collection set.

In [48]:
def obtain_image_sentinel_allbands(collection, time_range, area):
    """ Query satellite imagegry from GEE 
    - [B2, B3, B4, B8] - 10m resolution
    - use images only with <= 10% cloud-cover 
    from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Args:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
         (ee.ImageCollection)
     """

    sentinel_filtered = (ee.ImageCollection(collection).
                       filterBounds(area))
                       #filterDate(time_range.start(), time_range.end()))
    

    return sentinel_filtered

In [206]:
collection_sentinel = ('COPERNICUS/S2_SR')
time_range = ee.DateRange('2019-08-01', '2020-05-31')
aoi_geom = ee.Geometry(aoi.first().getInfo()['geometry'])
images_collection = obtain_image_sentinel_allbands(collection_sentinel, time_range, area=aoi_geom)
#images_collection = obtain_image_sentinel_allbands(collection_sentinel, time_range, area=aoi)

In [207]:
images_collection.size().getInfo()
#images_collection.getInfo()['Features'][1]

497

{'crs': 'EPSG:32643',
 'transform': [10, 0, 600000, 0, -10, 1700040],
 'type': 'Projection'}

In [54]:
def temporal_composite(duration, reducer):
  # method to remove cloud from the image
  def maskclouds(image):
      """To mask clouds using the Sentinel-2 QA band
      @param {ee.Image} image Sentinel-2 image
      @return {ee.Image} cloud masked Sentinel-2 image
      """
      band_qa = image.select('QA60')
      cloud_mask = ee.Number(2).pow(10).int()
      cirrus_mask = ee.Number(2).pow(11).int()
      mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
          band_qa.bitwiseAnd(cirrus_mask).eq(0))
      return image.updateMask(mask).divide(10000)
  
  def fndvi(image):
      """Add NDVI to each pixel in an image"""
      #TODO: ask toby about this NDVI calculation
      # since its 20m resolution
      # there are other NDVIs at 10m resolution
      ndvi = image.expression(
      "(NIR-RED)/(NIR+RED)",
      {
          'RED': image.select('B4').multiply(0.0001),
          'NIR' : image.select('B5').multiply(0.0001)
      
      });# okay
      ndf = ndvi.rename('FNDVI')
      results = ndf.copyProperties(image, ['system:time_start'])
      return image.addBands(results)
  return (images_collection.filterDate(duration.start(), duration.end()).
          filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).
          map(maskclouds).
          map(fndvi).
          select(['B.*', 'FNDVI']).
          reduce(reducer))


In [55]:
start1 = ee.Date('2019-12-01')
end1 = start1.advance(1, 'month')
duration = ee.DateRange(start1, end1)
month1 = temporal_composite(duration, ee.Reducer.max())


In [217]:
def create_monthly_max_composite(month_no):
  month_start_date = start_date.advance(month_no, 'month')
  month_end_date = month_start_date.advance(1, 'month')
  duration = ee.DateRange(start_date, month_end_date)

  image = (temporal_composite(duration, ee.Reducer.max()).
           set('system:time_start', month_start_date.millis()).
           set('system:id', month_start_date.format('YYYYMM')).
           set('system:index', month_start_date.format('YYYYMM')))

  def new_name(old_name):
    return month_name.cat(old_name)
  month_name = month_start_date.format("YYYY'_'MMM'_'")
  band_names = image.bandNames()
  new_band_names = band_names.map(new_name)
  image = image.rename(new_band_names)
  return image.clip(aoi)


In [218]:
month1 = create_monthly_max_composite(0)

In [219]:
month1.getInfo()

{'bands': [],
 'id': '201908',
 'properties': {'system:index': '201908', 'system:time_start': 1564617600000},
 'type': 'Image'}

In [171]:
monthly_collection.aggregate_array('system:index').getInfo()

['20190801',
 '20190901',
 '20191001',
 '20191101',
 '20191201',
 '20200101',
 '20200201',
 '20200301',
 '20200401',
 '20200501']

In [222]:
def get_composites_for_aoi(aoi):
  collection_sentinel = ('COPERNICUS/S2_SR')
  time_range = ee.DateRange('2019-08-01', '2020-05-31')
  start_date = time_range.start()
  end_date = time_range.end()
  count = end_date.difference(start_date, 'month').round().toInt()
  month_seq = ee.List.sequence(0, count.subtract(1))
 
  aoi_geom = ee.Geometry(aoi.getInfo()['geometry'])
  images_collection = obtain_image_sentinel_allbands(collection_sentinel, time_range, area=aoi_geom)

  projection = images_collection.first().select('B2').projection().getInfo()
  monthly_collection = ee.ImageCollection.fromImages(month_seq.map(create_monthly_max_composite))

  out_dir='/content/drive/MyDrive/FarmHand/data/sentinel_composites'

  geemap.ee_export_image_collection(monthly_collection, out_dir=out_dir, region=aoi_geom)

In [225]:
aoi.first().getInfo()

{'geometry': {'coordinates': [[[76.65, 15.2],
    [76.8, 15.2],
    [76.8, 15.37],
    [76.65, 15.37],
    [76.65, 15.2]]],
  'type': 'Polygon'},
 'id': '0',
 'properties': {},
 'type': 'Feature'}

In [223]:
 get_composites_for_aoi(aoi.first())

Total number of images: 10

Exporting 1/10: 201908.tif
Generating URL ...
An error occurred while downloading.
Image.projection: Can't get band number 0. Image has no bands.


Exporting 2/10: 201909.tif
Generating URL ...
An error occurred while downloading.
Image.projection: Can't get band number 0. Image has no bands.


Exporting 3/10: 201910.tif
Generating URL ...
An error occurred while downloading.
Image.projection: Can't get band number 0. Image has no bands.


Exporting 4/10: 201911.tif
Generating URL ...


KeyboardInterrupt: ignored

In [164]:
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [179]:
type(aoi.geometry())

ee.geometry.Geometry

Total number of images: 10

Exporting 1/10: 20190801.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FarmHand/data/sentinel_composites/20190801.tif


Exporting 2/10: 20190901.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FarmHand/data/sentinel_composites/20190901.tif


Exporting 3/10: 20191001.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FarmHand/data/sentinel_composites/20191001.tif


Exporting 4/10: 20191101.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FarmHand/data/sentinel_composites/20191101.tif


Exporting 5/10: 20191201.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FarmHand/data/sentinel_composites/20191201.tif


Exporting 6/10: 20200101.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FarmHand/data/sentinel_composites/20200101.tif


Exporting 7/10: 20200201.tif
Generating UR

## Download Data

In [ ]:
count = int(images_collection.size().getInfo())
print(f"Total number of images: {count}\n")
download_dir = '/content/sentinel_download'
for i in range(0, count): #TODO change number to count
    image = ee.Image(images_collection.toList(count).get(i))
    name = image.get("system:index").getInfo() + ".tif"
    #filename = f'drive/MyDrive/farmhand/sentinel_images/{name}' 
    filename = f"{download_dir}/{name}"
    print(f"Exporting {i + 1}/{count}: {name}")
    # scale = image.projection().nominalScale().multiply(10)
    # download_url = image.getDownloadURL({'region':aoi, 'scale':scale, "name": name, "filePerBand": True})
    # print(download_url)
    geemap.ee_export_image(
        image.select(['B2', 'B3', 'B4', 'B8']).clip(aoi),
        filename=filename,
        region=aoi,
        file_per_band=True
    )
    geemap.ee_export_image(
        image.select(['FNDVI']).clip(aoi),
        filename=filename,
        region=aoi,
        file_per_band=True
    )


# Visualizing Satellite Imagery

[Indices & composities](https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/false_color_infrared)

[Gee guide for visualization parameters](https://tutorials.geemap.org/Image/image_visualization/)

In [58]:

Map = geemap.Map(location=(15.3510, 76.7318), zoom=10)
vis_params = {
    'bands': ['B11_max', 'B8_max', 'B2_max'],
    'min': 0.2,
    'max': 1.0
}
vis_type = 'agriculture(B11,B8,B2)'

#image = ee.Image(images_collection.toList(100).get(10)) # map one image from collection
Map.addLayer(month1.clip(aoi), vis_params, vis_type) 
Map.addLayer(aoi, {'color': 'green'}, 'AOI', opacity=0.5)

# prepare farms polygons for plotting
#farms_gdf = gpd.GeoDataFrame({'geometry': df['geometry'].values}, crs=df.crs)
farm_geom = geemap.geopandas_to_ee(df)

Map.addLayer(farm_geom, {'color': 'cyan'}, opacity=0.5)
Map


Map(center=[15.351, 76.7318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…